In [1]:
import os
from glob import glob
import shutil
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile as tff
from skimage.io import imsave


In [2]:
ctlPath = '/Users/zhengj10/Desktop/210513/Control'
tgbPath = '/Users/zhengj10/Desktop/210513/Tiagabine'


In [3]:
# imgFiles = glob(imgPath+'/*.mat')
# imgFiles = sorted(imgFiles)
# len(imgFiles)
imgFile = tgbPath+'/FOV_00001.mat'


In [4]:
imgFile = loadmat(imgFile)

In [5]:
imgData = imgFile['data'][:,0][500:2900]


In [6]:
imgData_aslist = []
for i in range(imgData.shape[0]):
    imgData_aslist.append(imgData[i])
imgData = np.array(imgData_aslist)
imgData.shape

(2400, 512, 512)

In [8]:
from skimage import measure

imgFiles = glob('/Users/zhengj10/Desktop/test/'+'*.mat')
imgFiles = sorted(imgFiles)
len(imgFiles)
for fov in range(len(imgFiles)):
    print(imgFiles[fov])
    imgData = loadmat(imgFiles[fov])
    imgData = imgData['data']
    imgData_aslist=[]
    for frame in range(500,2900):
        imgData_aslist.append(imgData[:,0][frame])
    imgData = np.array(imgData_aslist)
    print(imgData.shape)
    img_norm = (imgData/imgData[0])-1

    act_pix = []
    for i in range(10,2390):
        a = (np.mean(img_norm[i+1:i+10,:,:],axis=0)-np.mean(img_norm[i-10:i,:,:],axis=0))>0.04
        act_pix.append(a)
    act_pix = np.array(act_pix)
    act_pix = np.max(act_pix,axis=0)

    contours = measure.find_contours(act_pix, 0.1)
    thre = 20 # Exclude small ROIs
    rois = []
    
    coordinates = []
    for r in range(act_pix.shape[0]):
        for c in range(act_pix.shape[1]):
            coordinates.append([r,c])
    coordinates = np.array(coordinates)
    
    
    for contour in contours:
        if contour.shape[0]>thre:

#             plt.plot(contour[:,1], contour[:, 0], color = 'lime',lw=1)
            p = Path(contour)
            roi = p.contains_points(coordinates)
            roi = roi.reshape(512,512)
            rois.append(roi)

    if len(rois) == 0:
        print('FOV_'+str(fov+1).zfill(3)+': no active pixels')
        continue
    fig = plt.figure(figsize=[5,5])
    
    plt.imshow(act_pix)
    for contour in contours:
        if contour.shape[0]>thre:
            plt.plot(contour[:,1], contour[:, 0], color = 'lime',lw=1)
            
    plt.axis('off')
    plt.tight_layout()
#     fig.savefig('qqq.pdf',format='pdf')
    print('FOV_'+str(fov+1).zfill(3)+' ROI image saved')
    fig.clear()
    plt.close(fig)

    Fb = np.min(imgData[-1,:,:])
    F0 = np.mean(imgData[:10,:,:],axis=0)
    
    fig = plt.figure(figsize=[12,len(rois)/2.0+0.1])

    offset_step = 0
    for roi in range(len(rois)):
        offset = 0.2*offset_step
        dFF_all = []
        for frame in range(imgData.shape[0]):
            F_single = np.sum(rois[roi]*imgData[frame,:,:])/np.sum(rois[roi])
            dFF = (F_single - np.sum(rois[roi]*F0)/np.sum(rois[roi]))/(np.sum(rois[roi]*F0)/np.sum(rois[roi])-Fb)
            dFF_all.append(dFF)
        dFF_all = np.array(dFF_all)
        plt.plot(dFF_all - offset, zorder=roi+1, color='k', lw=1,alpha=0.8)
        plt.axis('off')
        offset_step += 1
    plt.plot([2350,2450,2450],[-0.2*offset_step-0.1,-0.2*offset_step-0.1,-0.2*offset_step+0.1],color='k',alpha=0.8)
    plt.tight_layout()
#     fig.savefig(sumPath+'\\dFF_FOV_'+str(fov+1).zfill(3)+'.pdf',format='pdf')
    fig.clear()
    plt.close(fig)


/Users/zhengj10/Desktop/test/FOV_00001.mat
(2400, 512, 512)
FOV_001: no active pixels


In [ ]:
import pickle

In [ ]:
sumPath = os.path.join(ctlPath,'summary')
with open(sumPath+'/act_pixel.pickle', 'rb') as handle:
    act_pix = pickle.load(handle)


In [ ]:
act_pix_dict = {}

In [ ]:
for k,v in act_pix.items():
    act_pix_dict[k[-13:]] = v

In [ ]:
act_pix_dict

In [ ]:
act_pix = act_pix_dict['FOV_00001.mat'][0]

In [ ]:
act_pix

In [ ]:
# imgData = imgData[500:2900,:,:]
# img_norm = (imgData/imgData[0])-1

# act_pix = []
# for i in range(10,2390):
#     a = (np.mean(img_norm[i+1:i+10,:,:],axis=0)-np.mean(img_norm[i-10:i,:,:],axis=0))>0.04
#     act_pix.append(a)
# act_pix = np.array(act_pix)
# act_pix = np.max(act_pix,axis=0)
# plt.axis('off')
# plt.tight_layout()
plt.figure(figsize=[9,9])
plt.imshow(act_pix,cmap='gray')


In [ ]:
fig = plt.figure(figsize=[8,8])
plt.imshow(act_pix,cmap='gray')
plt.axis('off')
plt.tight_layout()
# fig.savefig(imgPath+'act_pix.png')

In [ ]:
from skimage import measure
contours = measure.find_contours(act_pix, 0.1)

In [ ]:
fig = plt.figure(figsize=[8,8])
plt.imshow(act_pix,cmap='gray')
thre = 20 # Exclude small ROIs
# roi = 0
for contour in contours:
    if contour.shape[0]>thre:
#         roi+=1
        plt.plot(contour[:,1], contour[:, 0], color = 'r',lw=5)
plt.axis('off')
plt.tight_layout()
# fig.savefig(imgPath+'roi.png')


In [ ]:
act_pix.shape

In [ ]:
for contour in contours:
    print(contour.shape)

In [ ]:
from matplotlib.path import Path
mean_image = imgData[0,:,:]
# plt.figure(figsize=[12,12])
plt.imshow(np.zeros([512,512]))
coordinates = []
for r in range(act_pix.shape[0]):
    for c in range(act_pix.shape[1]):
        coordinates.append([r,c])
coordinates = np.array(coordinates)

meanImg_b0 = []
rois = []
for contour in contours:
    if contour.shape[0]>thre:
        p = Path(contour)
        roi = p.contains_points(coordinates)
        roi = roi.reshape(512,512)
        f0 = np.sum(roi*mean_image)/np.sum(roi)
        rois.append(roi)
        meanImg_b0.append(roi*mean_image)


x = np.arange(0, 20)
y = np.arange(22, 32)
xv, yv = np.meshgrid(x, y)
mask = np.zeros(mean_image.shape)
mask[yv.flatten().astype(np.int), xv.flatten().astype(np.int)] = 1
rois.append(mask) # Add mask (fb)
meanImg_b0.append(rois[-1]*mean_image) # Add mask (fb)
rois_all = np.sum(rois,axis=0)      
        
plt.imshow(rois_all)
meanImg_b0 = np.sum(meanImg_b0,axis=0)
# plt.colorbar()
plt.show()


In [ ]:
# plt.figure(figsize=[12,12])
plt.imshow(meanImg_b0)
# plt.colorbar()
plt.show()


In [ ]:
len(rois)
# imgData.shape

In [ ]:
Fb = np.min(imgData[-1,:,:])
F0 = np.mean(imgData[:10,:,:],axis=0)
fig = plt.figure(figsize=[12,(len(rois)-1)/15])
i = 1
x = np.arange(1,2401)
for roi in range(len(rois)-1):
    if roi in [1,2]:
        continue
    offset = 0.05*i
    dFF_all = []
    for frame in range(imgData.shape[0]):
        F_single = np.sum(rois[roi]*imgData[frame,:,:])/np.sum(rois[roi])
        dFF = (F_single - np.sum(rois[roi]*F0)/np.sum(rois[roi]))/(np.sum(rois[roi]*F0)/np.sum(rois[roi])-Fb)
        dFF_all.append(dFF)
    dFF_all = np.array(dFF_all)
    dFF_all = signal.detrend(dFF_all)
    b, a = signal.butter(8, 0.4)
    dFF_all = signal.filtfilt(b, a, dFF_all, padlen=125)
    plt.fill_between(x,dFF_all - offset, 0 - offset, 
                 zorder=i+1, 
                 facecolor='w', edgecolor='w', 
                 lw=0, 
                 closed=True)
    plt.plot(x,dFF_all - offset, zorder=i+1, color='k', lw=0.5)
    plt.axis('off')
    i+=1
# plt.plot([2350,2450,2450],[-1.1,-1.1,-0.9],color='k',alpha=0.8)
plt.tight_layout()
# fig.savefig('/Users/zhengj10/Desktop/dFF_013.pdf',format='pdf')21


In [ ]:
from scipy import signal

In [ ]:
b, a = signal.butter(8, 0.4)
y = signal.filtfilt(b, a, dFF_all, padlen=125)
plt.plot(y)
plt.show()

In [ ]:
from scipy import signal

dFF_all_d = signal.detrend(dFF_all)

In [ ]:
fig = plt.figure(figsize=[15,12])
subplot_no=1
imgData_act_pix_all = {}
for imgFile in imgFiles:
    print(imgFile)
    imgData_act_pix_all[imgFile] = []
    imgData = loadmat(imgFile)
    imgData = imgData['data']
    imgData = imgData[:,0][500:2900]
    imgData_aslist = []
    for i in range(imgData.shape[0]):
        imgData_aslist.append(imgData[i])
    imgData = np.array(imgData_aslist)
    img_norm = (imgData/imgData[0])-1
    act_pix = []
    for i in range(10,2390):
        a = (np.mean(img_norm[i+1:i+10,:,:],axis=0)-np.mean(img_norm[i-10:i,:,:],axis=0))>0.04
        act_pix.append(a)
    act_pix = np.array(act_pix)
    act_pix = np.max(act_pix,axis=0)
    ax = plt.subplot(4,5,subplot_no)
    ax.imshow(act_pix,cmap='gray')
    imgData_act_pix_all[imgFile].append(act_pix)
    subplot_no+=1
# fig.savefig('/Users/zhengj10/Desktop/210203/act_pix.png')

In [ ]:
fig = plt.figure(figsize=[15,12])
subplot_no=1
for imgFile in imgFiles:
    print(imgFile)
    imgData = loadmat(imgFile)
    imgData = imgData['data']
    imgData = imgData[:,0][500]
    ax = plt.subplot(4,5,subplot_no)
    ax.imshow(imgData,cmap='gray',vmin=1500,vmax=5000)
    ax.axis('off')

    subplot_no+=1
plt.tight_layout()
# fig.savefig(imgPath + 'all_FOVs.png')

In [ ]:
import pickle

with open('act_pixel.pickle', 'wb') as handle:
    pickle.dump(imgData_act_pix_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('act_pixel.pickle', 'rb') as handle:
#     b = pickle.load(handle)

In [ ]:
img_1 = np.zeros((32,32))
img_2 = np.zeros((32,32))

In [ ]:
img_1[12,15] = 5
img_2[15,12] = 2
plt.imshow(img_1,cmap='gray')
plt.imshow(img_2,cmap='gray',alpha=0.5)